## 3.4 Dünn besetzte Matrizen

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

Um dünnbesetzte Matrizen auch als solche zu speichern, verwenden wir dir Bibliothek `scipy` die auch im wissenschaftlichen Rechnen Verwendung findet. Des Weiteren nutzen wir `matplotlib` um die Besetzungsstruktur der Matrizen zu visualisieren.

**Implementierung 3.6: Cuthill-McKee-Algorithmus**

In [ ]:
def cuthill_mckee(A):
    n = A.shape[0]
    row, col = A.nonzero()
    N = [(l, len(l)) for l in [list(row[col == i]) for i in range(n)]]

    I, Q = [], []
    R = [i for i in range(n)]

    for k in range(n):
        if len(I) == n:
            break
        elif len(I) == k:
            i = R[np.argmin(np.array([N[i][1] for i in R]))]
            I.append(i)
            Q.append(i)
            R.remove(i)

        i = Q[0]
        nachbarn = [n for n in N[i][0] if n not in I]
        nachbarn_sort = sorted(nachbarn, key=lambda i: N[i][1])
        for ik in nachbarn_sort:
            I.append(ik)
            Q.append(ik)
            R.remove(ik)
        Q.pop(0)

    data, row, col = [], [], []
    for key in A.todok().keys():
        data.append(A[key])
        row.append(I.index(key[0]))
        col.append(I.index(key[1]))

    return sp.sparse.csr_matrix((data, (row, col)), shape=(n, n))

#### Beispiel 3.24 (Cuthill-McKee)
Wir wenden unsere Implementierung auf die im Buch gegebene Matrix an. Dabei nehmen wir einfachheitshalber 1 als nicht-null Wert.

In [ ]:
A1 = sp.sparse.csr_matrix([[1, 1, 0, 0, 0, 0, 0, 1],
                           [1, 1, 0, 0, 1, 1, 0, 1],
                           [0, 0, 1, 0, 1, 0, 1, 1],
                           [0, 0, 0, 1, 0, 0, 0, 0],
                           [0, 1, 1, 0, 1, 0, 1, 0],
                           [0, 1, 0, 0, 0, 1, 0, 0],
                           [0, 0, 1, 0, 1, 0, 1, 0],
                           [1, 1, 1, 0, 0, 0, 0, 1]])

plt.spy(A1)
plt.show()

In [ ]:
A2 = cuthill_mckee(A1)
plt.spy(A2)
plt.show()

#### Beispiel 3.25 (Cholesky-Zerlegung für Bandmatrizen)

Aus Satz 3.22 (LR-Zerlegung einer Bandmatrix) wissen wir, dass die LR-Zerlegung einer Bandmatrix wieder Bandmatrizen mit derselben Bandbreite produziert. Dies können wir in der Implementierung der Cholesky-Zerlegung ausnutzen, um unnötige Arbeit zu vermeiden.

In [ ]:
def cholesky_band(A):
    n, m = A.shape
    assert n == m, 'Matrix must be square!'
    L = np.zeros_like(A)

    for p in range(n - 1):
        if np.allclose(L, np.triu(A, k=p + 1)) and np.allclose(L, np.tril(A, k=-p - 1)):
            break

    for j in range(0, n):
        ll = 0
        m = max(0, j - p)
        for k in range(m, j):
            ll += L[j, k]**2
        L[j, j] = np.sqrt(A[j, j] - ll)

        for i in range(j + 1, min(j + 1 + p, n)):
            ll = 0
            for k in range(m, j):
                ll += L[i, k] * L[j, k]
            L[i, j] = (A[i, j] - ll) / L[j, j]
    return L

Um dies zu testen, generieren wir zunächst eine dünnbesetzte symmetrisch positiv definite Matrix.

In [ ]:
C = sp.sparse.rand(100, 100, density=0.02, format='csr', random_state=100)
X = sp.sparse.eye(100) + C @ C.T

plt.tick_params(axis='both', which='both', labelbottom=False, labelleft=False, width=0)
plt.spy(X, markersize=2)
plt.show()

Wenden wir die Cholesky-Zerlegung direkt an, so sehen wir, dass die Matrix $L$ im unteren Dreieck fast vollbesetzt ist. Insbesondere die unteren Reihen haben kaum Null-Einträge.

In [ ]:
L = cholesky_band(X.toarray())
plt.spy(sp.sparse.csr_matrix(L), markersize=2)
plt.show()

Wenden wir aber erst den Cuthill-McKee-Algorithmus auf die Matrix an, bekommen wir die Matrix in einer Bandstruktur, die wir in der Cholesky-Zerlegung ausnutzen können.

In [ ]:
X1 = cuthill_mckee(X)
plt.spy(X1, markersize=2)
plt.show()

Die Cholesky-Zerlegung nutzt dies dann aus und wir erhalten in $L$ wieder eine dünnbesetzte Matrix:

In [ ]:
L = cholesky_band(X1.toarray())
plt.spy(sp.sparse.csr_matrix(L), markersize=2)
plt.show()

Wir beobachten allerdings, dass innerhalb der Bandbreite durchaus Nicht-Null-Einträge in die Besetzungsstruktur hinzu kommen. Im Vor- und Rückwärtseinsetzen können wir nun aber auch die Bandstruktur ausnutzen. 